# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

<div class="alert alert-success" style="border-radius: 15px; box-shadow: 4px 4px 4px; border: 1px solid ">
<h2> Комментарий ревьюера</h2>
    
Правильно делаешь, что не убираешь краткое описание задачи. Это поможет быстро вспомнить, о чем проект, если ты к нему вернешься спустя какое-то время.
</div>

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')


Получаем информацию по датасету.

In [2]:
df.info()
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Датасет содержит всю необходимую для исследования информацию. В важном для нас столбце "доход в месяц" есть пропуски, которые необходимо восстановить. Столбец "трудовой стаж" имеет пропущенные значение и данные с отрицательными значениями, которые не возможно интерпретировать.

<div class="alert alert-success" style="border-radius: 15px; box-shadow: 4px 4px 4px; border: 1px solid ">
<h2> Комментарий ревьюера</h2>
    
Причем, по модулю числа в столбце стажа кажутся нормальными. Так что минус мог быть просто тире. А вот огромные вызывают много вопросов. Это могут быть часы, а могут быть и пропуски, которые при выгрузке трансформировались в аномальные для конкретного столбца числа.
</div>

## Шаг 2. Предобработка 

### Обработка пропусков

Проверим на пропуски.

In [3]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Столбец 'days_employed' удалим, так как для нашего исследования он не нужен.

In [4]:
del df['days_employed'] 

Для заполнения пропусков в столбце доход в месяц, посчитаем медианный доход по категории занятости.

In [5]:
median_total_income = df.groupby('income_type')['total_income'].transform('median')

<div class="alert alert-success" style="border-radius: 15px; box-shadow: 4px 4px 4px; border: 1px solid ">
<h2> Комментарий ревьюера</h2>
    
Отличный код! При желании, можно будет легко заполнить пропуски в группировке по 2-3 признакам. </div>

Заменим пропущенные значения.

In [6]:
df['total_income'].fillna(median_total_income, inplace=True)

Проверияем наличие пропусков.

In [7]:
df.isna().sum()
df.tail(15)

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21510,2,28,среднее,1,женат / замужем,0,F,сотрудник,0,142594.396847,приобретение автомобиля
21511,0,29,высшее,0,гражданский брак,1,F,сотрудник,1,140068.472941,покупка жилья для сдачи
21512,0,26,высшее,0,Не женат / не замужем,4,M,компаньон,0,147301.457769,получение дополнительного образования
21513,0,35,среднее,1,женат / замужем,0,F,сотрудник,0,250986.142309,покупка жилья
21514,0,27,неоконченное высшее,2,Не женат / не замужем,4,M,компаньон,0,355988.407188,строительство недвижимости
21515,1,28,среднее,1,женат / замужем,0,F,сотрудник,1,109486.327999,заняться образованием
21516,0,42,высшее,0,женат / замужем,0,F,компаньон,0,322807.776603,покупка своего жилья
21517,0,42,высшее,0,гражданский брак,1,F,компаньон,0,178059.553491,на покупку своего автомобиля
21518,0,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем
21519,1,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949.039788,покупка коммерческой недвижимости


В столбцах 'gender', 'children', 'dob_years' есть не допустимые значения.

In [8]:
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

В столбце "пол" есть лишнее значение, удалим его. 

In [9]:
df = df.drop(df[df['gender'] == 'XNA'].index)

В столбце 'children' есть опечатки.

In [10]:
df['children'].value_counts()

 0     14148
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [11]:
# Заменим -1 на 1, 20 на 2
df['children'] = df['children'].replace(-1, 1)
df['children'] = df['children'].replace(20, 2)

<div class="alert alert-success" style="border-radius: 15px; box-shadow: 4px 4px 4px; border: 1px solid ">
<h2> Комментарий ревьюера</h2>
    
Ну да. Учитывая отсутствие клиентов с 6-19 детьми, а так же число 76, я склоняюсь к тому, что это ошибка. Скорее всего там либо 2, либо 20 это маркер пропущенного значения. </div>

Столбец 'dob_years' имеет явно ошибочное значение "0", установить правильное значение нет возможности, поэтому удалим его. 

In [12]:
df['dob_years'].value_counts()

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    263
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

<font color='blue'> Я изначально тоже сомневался, удалять или нет. Но с вами согласен 101 человек может добавить статистики. </font>  

Для обработки пропусков: удалили столбец 'days_employed'; в столбце 'total_income' посчитали медианный доход по типу занятости; в столбцах 'gender', 'children', 'dob_years' удалили не допустимые значения.

<div class="alert alert-warning" style="border-radius: 15px; box-shadow: 4px 4px 4px; border: 1px solid ">
<h2> Комментарий ревьюера</h2>
    
А надо ли их удалять? Ну, нули и ладно. Не мешают же :)</div>

### Замена типа данных

Столбец 'total_income' имеет тип float64, переведем в тип int64. 

In [13]:
df['total_income'] = df['total_income'].astype('int')

In [14]:
df.head(15)

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


Чтобы получить более читаемый вид, изменили тип 'total_income' с float на int.

### Обработка дубликатов

Проверяем количество дубликатов

In [15]:
df.duplicated().sum()

54

Проверяем уникальные значения в столбцах. 

In [16]:
df['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Приводим стоки 'education' к одному регистру.

In [17]:
df['education'] = df['education'].str.lower()
df['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      743
начальное                282
ученая степень             6
Name: education, dtype: int64

Удаляем дубликаты

In [18]:
df = df.drop_duplicates().reset_index(drop=True)

<div class="alert alert-success" style="border-radius: 15px; box-shadow: 4px 4px 4px; border: 1px solid ">
<h2> Комментарий ревьюера</h2>
    
Поскольку у нас нет уникальных идентификаторов пользователей, полные дубликаты вполне могут быть разными людьми. А вот дубликаты записей в столбцах вроде **Среднее** образование и **среднее** образование удалить надо было (путем приведения к одному регистру), это верно. </div>

In [19]:
df.duplicated().sum()

0

В столбце 'education' названия категорий имеели разный регист записи. 

### Лемматизация

В столбце 'purpose' цели кредита целые предложения, сокротим для лучшей читаемости.

Импортируем библиатеку pymystem3

In [20]:
from pymystem3 import Mystem
m = Mystem()                # присваеваем m функцию Mystem()

Посмотрим на какие цели берут кредит.

In [21]:
df['purpose'].value_counts() 

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      620
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Из общего списка целей выделяем ключевые слова: жилье, недвижимость, свадьба, автомобиль, образование. 

<div class="alert alert-success" style="border-radius: 15px; box-shadow: 4px 4px 4px; border: 1px solid ">
<h2> Комментарий ревьюера</h2>
    
Согласна.</div>

Напишем функцию для поиска ключевого слова в строке столбца 'purpose'.

In [22]:
def lemmas(purpose):
    lemma = m.lemmatize(purpose)
    if ('жилье' in lemma) or ('недвижимость' in lemma):
        find_lemmas = 'жилье'
        return find_lemmas
    elif 'свадьба' in lemma:
        find_lemmas = 'свадьба'
        return find_lemmas
    elif 'автомобиль' in lemma:
        find_lemmas = 'автомобиль'
        return find_lemmas
    elif 'образование' in lemma:
        find_lemmas = 'образование'
        return find_lemmas

Присвоим соответсвующим предложениям короткие значения в столбце 'purpose'

In [23]:
purpose_lemma = df['purpose'].apply(lemmas)
df['purpose'] = purpose_lemma
df.head(10)

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,жилье
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,жилье
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба
5,0,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,жилье
6,0,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,жилье
7,0,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,свадьба
9,0,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,жилье


Получили более читаемую таблицу.

### Категоризация данных

Добавим две категории по возросту и доходам.

In [24]:
# Сортируем таблицу по возросту
df.sort_values('dob_years')

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
1890,0,0,высшее,0,Не женат / не замужем,4,F,сотрудник,0,142594,жилье
19769,0,0,среднее,1,женат / замужем,0,F,сотрудник,0,142594,жилье
13398,1,0,высшее,0,гражданский брак,1,F,сотрудник,0,91924,свадьба
13207,0,0,высшее,0,женат / замужем,0,M,сотрудник,0,131782,образование
11644,0,0,среднее,1,женат / замужем,0,F,компаньон,0,73338,образование
...,...,...,...,...,...,...,...,...,...,...,...
4891,0,74,высшее,0,женат / замужем,0,F,пенсионер,0,134935,жилье
12296,0,74,среднее,1,женат / замужем,0,M,компаньон,0,124830,жилье
3458,0,74,среднее,1,женат / замужем,0,M,пенсионер,0,54754,жилье
11512,0,74,среднее,1,гражданский брак,1,F,сотрудник,0,98945,свадьба


Возрост от 19 до 75 лет. Разделим на 3 возростные группы.

In [25]:
def dob_years_grouped(age):
    if 19 <= age <= 35: return 'молодые'
    elif 36 < age < 59: return 'взрослые'
    return 'пенсионеры'

In [26]:
dob_years_grouped = df['dob_years'].apply(dob_years_grouped)
df['dob_years'] = dob_years_grouped
df

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,взрослые,высшее,0,женат / замужем,0,F,сотрудник,0,253875,жилье
1,1,пенсионеры,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль
2,0,молодые,среднее,1,женат / замужем,0,M,сотрудник,0,145885,жилье
3,3,молодые,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование
4,0,взрослые,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба
...,...,...,...,...,...,...,...,...,...,...,...
21448,1,взрослые,среднее,1,гражданский брак,1,F,компаньон,0,224791,жилье
21449,0,пенсионеры,среднее,1,женат / замужем,0,F,пенсионер,0,155999,автомобиль
21450,1,взрослые,среднее,1,гражданский брак,1,M,сотрудник,1,89672,жилье
21451,3,взрослые,среднее,1,женат / замужем,0,M,сотрудник,1,244093,автомобиль


In [27]:
# Сортируем таблицу по доходу
df.sort_values('total_income')

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
14554,0,взрослые,среднее,1,женат / замужем,0,F,пенсионер,1,20667,жилье
12982,0,взрослые,среднее,1,гражданский брак,1,M,пенсионер,0,21205,образование
16137,1,взрослые,среднее,1,женат / замужем,0,M,сотрудник,0,21367,автомобиль
1598,0,пенсионеры,среднее,1,гражданский брак,1,M,пенсионер,0,21695,свадьба
14246,0,пенсионеры,среднее,1,женат / замужем,0,F,пенсионер,0,21895,жилье
...,...,...,...,...,...,...,...,...,...,...,...
17136,0,взрослые,высшее,0,гражданский брак,1,M,компаньон,0,1711309,свадьба
20741,0,пенсионеры,среднее,1,Не женат / не замужем,4,F,сотрудник,0,1715018,жилье
9159,1,молодые,среднее,1,гражданский брак,1,M,сотрудник,0,1726276,образование
19547,1,взрослые,высшее,0,женат / замужем,0,M,компаньон,1,2200852,жилье


Уровень доходов от 20 т. руб. до 2.26 млн. руб.

<font color='blue'> Поменял для справедливости, хотя и не согласен. </font>  

In [28]:
# Создадим функцию для разделениям по категориям
def total_income_grouped(income):
    if 20000 <= income <= 70000: return 'низкий'
    elif 70000 < income <= 120000: return 'средний'
    elif 120000 < income <= 200000: return 'высокий'
    return 'очень высокий'

In [29]:
total_income_grouped = df['total_income'].apply(total_income_grouped)
df['total_income'] = total_income_grouped
df

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,взрослые,высшее,0,женат / замужем,0,F,сотрудник,0,очень высокий,жилье
1,1,пенсионеры,среднее,1,женат / замужем,0,F,сотрудник,0,средний,автомобиль
2,0,молодые,среднее,1,женат / замужем,0,M,сотрудник,0,высокий,жилье
3,3,молодые,среднее,1,женат / замужем,0,M,сотрудник,0,очень высокий,образование
4,0,взрослые,среднее,1,гражданский брак,1,F,пенсионер,0,высокий,свадьба
...,...,...,...,...,...,...,...,...,...,...,...
21448,1,взрослые,среднее,1,гражданский брак,1,F,компаньон,0,очень высокий,жилье
21449,0,пенсионеры,среднее,1,женат / замужем,0,F,пенсионер,0,высокий,автомобиль
21450,1,взрослые,среднее,1,гражданский брак,1,M,сотрудник,1,средний,жилье
21451,3,взрослые,среднее,1,женат / замужем,0,M,сотрудник,1,очень высокий,автомобиль


<div class="alert alert-info">
<h2> Комментарий студента</h2>

Если взять нашу страну, то доход в 50 000 р., это да же выше среднего.
Людям с низким доходом чаще отказывают в выдаче крупных кредитов, а с высоким наоборот чаще выдают.
Если бы мы взяли микрозаймы, то там скорей всего инстория была бы другая. Но у нас другие цели, а там суммы значительные.
</div>


<font color='blue'> </font>     

<div class="alert alert-success" style="border-radius: 15px; box-shadow: 4px 4px 4px; border: 1px solid ">
<h2> Комментарий ревьюера 2</h2>
    
Тут еще вот какая ситуация: мы не можем быть уверены, что речь о нашей стране. Очень странная выборка.  Совсем не ясно, на какие группы делить, поэтому лучше отталкиваться от распределения, чтобы эти группы хотя бы можно было сравнивать между собой. Могу предложить в таких ситуациях просто больше категорий делать.</div>
<div class="alert alert-warning" style="border-radius: 15px; box-shadow: 4px 4px 4px; border: 1px solid ">
<b> Комментарий ревьюера</b>
    
Не самое честное распределение. Людей с высоким доходом 16 тысяч, 4 тысячи со средним и всего 370 человек с низким. </div>

In [30]:
# КОД РЕВЬЮЕРА

df['total_income'].value_counts()

высокий          9156
средний          5757
очень высокий    5066
низкий           1474
Name: total_income, dtype: int64

Для дальнейшего исследования сгрупперовали по возросту и доходам. 

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [31]:
# вызовем функцию для зависимости дети/просрочка
df.groupby('children').agg({'debt':['count', 'sum']})

debt      
          count   sum
children             
0         14090  1063
1          4855   445
2          2128   202
3           330    27
4            41     4
5             9     0

<font color='blue'> Спасибо за подсказку. Улучшил код. </font>  

In [32]:
df.groupby('children')['debt'].mean()

children
0    0.075444
1    0.091658
2    0.094925
3    0.081818
4    0.097561
5    0.000000
Name: debt, dtype: float64

<div class="alert alert-success" style="border-radius: 15px; box-shadow: 4px 4px 4px; border: 1px solid ">
<h2> Комментарий ревьюера</h2>
    
Обрати внимание на то, что ты делишь сумму на количество, а это не что иное, как среднее. Так что это все можно сделать в группировке:
</div>

In [33]:
# КОД РЕВЬЮЕРА

# Или просто форматируем вывод
df.groupby('children')['debt'].agg(['count', 'sum', lambda x: x.mean()])

,count,sum,<lambda_0>
children,,,
0,14090,1063,0.075444
1,4855,445,0.091658
2,2128,202,0.094925
3,330,27,0.081818
4,41,4,0.097561
5,9,0,0.000000


 Зависимость между наличием детей и возваратом кредита в срок есть. Чем больше детей, тем меньше берут кредит.

<div class="alert alert-info">
<h2> Комментарий студента</h2>

Больше всего просрочек у семей с 1-4 детьми, скорей всего из-за больших расходов на детей . Нет просрочек у семей с 5 детьми. Так как такие семьи практически не берут кредиты из-за финансовой нагрузки.
</div>




<div class="alert alert-danger" style="border-radius: 15px; box-shadow: 4px 4px 4px; border: 1px solid ">
<h2> Комментарий ревьюера</h2>
    
> Зависимость между наличием детей и возваратом кредита в срок есть.
    
    
    
> Чем больше детей, тем меньше берут кредит.
    
    
Хорошо, кредиты с детьми берут реже. А что с возвратами?    
</div>

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [34]:
df.groupby('family_status').agg({'debt':['count', 'sum']})

debt     
                       count  sum
family_status                    
Не женат / не замужем   2810  274
в разводе               1195   85
вдовец / вдова           959   63
гражданский брак        4150  388
женат / замужем        12339  931

In [35]:
df.groupby('family_status')['debt'].mean()

family_status
Не женат / не замужем    0.097509
в разводе                0.071130
вдовец / вдова           0.065693
гражданский брак         0.093494
женат / замужем          0.075452
Name: debt, dtype: float64

Больше всего просрочек у холостых и тех кто живет в гражданском браке. Меньше просрочек у вдовед/вдова. Гипотеза не подтвердилась.

<div class="alert alert-info">
<h2> Комментарий студента</h2>

Хуже всего показатель у "не женатых / не замужем", чаще всего это молодые люди до 30 лет, берут кредит не правильно оценивая свои издержки. Люди в "гражданском браке", в виду того что нет такого юдрического статутса, часто такие союзы распадаются и люди перестают платить кредиту.
Зависимоть есть, люди с семейным статусом лучше платят по кредиту.
</div>

<div class="alert alert-danger" style="border-radius: 15px; box-shadow: 4px 4px 4px; border: 1px solid ">
<b> Комментарий ревьюера</b>
    
> Гипотеза не подтвердилась.
    
    
Почему?    
</div>

<div class="alert alert-success" style="border-radius: 15px; box-shadow: 4px 4px 4px; border: 1px solid ">
<h2> Комментарий ревьюера 2</h2>
    
То есть зависимость есть, а значит, гипотеза о ее наличии подтвердилась :) </div>

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [36]:
df.groupby('total_income').agg({'debt':['count', 'sum']})

debt     
              count  sum
total_income            
высокий        9156  797
низкий         1474  101
очень высокий  5066  358
средний        5757  485

In [37]:
df.groupby('total_income')['debt'].mean()

total_income
высокий          0.087047
низкий           0.068521
очень высокий    0.070667
средний          0.084245
Name: debt, dtype: float64

Люди с низким доходом меньше всего допускают просрочку по кредиту, а с средним и высоким доходом допускают просрочку больше. 

- Как разные цели кредита влияют на его возврат в срок?

In [38]:
df.groupby('purpose').agg({'debt':['count', 'sum']})

debt     
             count  sum
purpose                
автомобиль    4306  403
жилье        10810  782
образование   4013  370
свадьба       2324  186

In [39]:
df.groupby('purpose')['debt'].mean()

purpose
автомобиль     0.093590
жилье          0.072340
образование    0.092200
свадьба        0.080034
Name: debt, dtype: float64

По кредитам на автомобиль и образование чаще допускают просрочку, а на покупку жилья просрочку допускают реже.

## Шаг 4. Общий вывод

<font color='blue'> Прошу прощения, почему то не сохранилось. </font>  

Из анализа полученных данных можно сделать вывод:

1. Люди бывшие в оциальных отношениях "вдова/вдовец" имеют наименьше процент просрочки - 6.5%. Похожий результат у людей в "разводе" - 7.2%. Семейные люди "женат/замужем" близки к тем кто в "разводе" 7.5%. Самый плохой показакель у "не женат/ не замужем" 9.7% и "гражданский брак" 9.3%.
2. Больше всего выдано кредитов людям в статусе "женат / замужем" 12290 раз, с долей просрочки в 7.5%. Можно предположить что цель кредита этой группы покупка жилья, так как доля просрочки в категории желье 7.2%. 
3. Вывод: человек в статусе "женат / замужем" желающий взять кред на покупку желья является самым привликательным для банка. А "не женат / не замужем" желающий взять кредит на покупку автомобиля или образование самым не надежным.


<div class="alert alert-danger" style="border-radius: 15px; box-shadow: 4px 4px 4px; border: 1px solid">
<b> Комментарий ревьюера</b>
    
Похоже, вывод не сохранился :) В идеале надо бы привести проценты невозврата в срок у самых надежных и самых безответственных клиентов для каждой группы. Например: самые надежные клиенты это люди, берущие кредит на операции с недвижимостью (такой-то процент невозврата в срок), а самые безответственные те, кто обращается в банк с целью получить деньги на покупку автомобиля (такой-то процент). И так для каждой группы. Это хороший тон, — подкреплять вывод полученными ранее результатами. Кроме того, так ты не оставляешь заказчику шансов неверно интерпретировать вывод.
</div>

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [ ]  файл изучен;
- [ ]  определены пропущенные значения;
- [ ]  заполнены пропущенные значения;
- [ ]  есть пояснение, какие пропущенные значения обнаружены;
- [ ]  описаны возможные причины появления пропусков в данных;
- [ ]  объяснено, по какому принципу заполнены пропуски;
- [ ]  заменен вещественный тип данных на целочисленный;
- [ ]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [ ]  удалены дубликаты;
- [ ]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [ ]  описаны возможные причины появления дубликатов в данных;
- [ ]  выделены леммы в значениях столбца с целями получения кредита;
- [ ]  описан процесс лемматизации;
- [ ]  данные категоризированы;
- [ ]  есть объяснение принципа категоризации данных;
- [ ]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [ ]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [ ]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [ ]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [ ]  в каждом этапе есть выводы;
- [ ]  есть общий вывод.